# 교차검증과 하이퍼파라미터 튜닝

이 노트북에서는 모델의 일반화 성능을 평가하는 교차검증과 최적의 하이퍼파라미터를 찾는 방법을 학습합니다.

## 목차
1. 교차검증 (Cross-Validation)
   - K-Fold Cross-Validation
   - Stratified K-Fold
   - Leave-One-Out CV
   - Time Series Split
2. 하이퍼파라미터 튜닝
   - Grid Search
   - Randomized Search
3. 고급 기법
   - Nested Cross-Validation
   - Learning Curves

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_breast_cancer, load_diabetes
from sklearn.model_selection import (
    cross_val_score, cross_validate,
    KFold, StratifiedKFold, LeaveOneOut, ShuffleSplit,
    TimeSeriesSplit, RepeatedKFold,
    GridSearchCV, RandomizedSearchCV,
    learning_curve, validation_curve
)
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, accuracy_score
from scipy.stats import uniform, randint

# 시각화 설정
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.family'] = 'AppleGothic'  # MacOS용 한글 폰트
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

## 1. 교차검증 (Cross-Validation)

### 1.1 K-Fold Cross-Validation

In [ ]:
# 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

# 모델 생성
model = LogisticRegression(max_iter=1000, random_state=42)

# K-Fold 교차검증 (K=5)
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

print("=== K-Fold 교차검증 (K=5) ===")
print(f"각 폴드 점수: {scores}")
print(f"평균 정확도: {scores.mean():.4f}")
print(f"표준편차: {scores.std():.4f}")
print(f"95% 신뢰구간: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

In [ ]:
# K-Fold 수동 구현으로 이해하기
kf = KFold(n_splits=5, shuffle=True, random_state=42)

print("\n=== K-Fold 분할 시각화 ===")
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
    # 데이터 분할
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # 모델 학습 및 평가
    model_fold = LogisticRegression(max_iter=1000, random_state=42)
    model_fold.fit(X_train, y_train)
    score = model_fold.score(X_val, y_val)
    fold_scores.append(score)
    
    print(f"Fold {fold}: Train={len(train_idx)}, Val={len(val_idx)}, Accuracy={score:.4f}")

print(f"\n평균 정확도: {np.mean(fold_scores):.4f}")

In [ ]:
# K 값에 따른 성능 비교
k_values = [3, 5, 10, 15, 20]
mean_scores = []
std_scores = []

for k in k_values:
    scores = cross_val_score(model, X, y, cv=k, scoring='accuracy')
    mean_scores.append(scores.mean())
    std_scores.append(scores.std())

# 시각화
plt.figure(figsize=(10, 6))
plt.errorbar(k_values, mean_scores, yerr=std_scores, marker='o', 
             capsize=5, capthick=2, linewidth=2)
plt.xlabel('K (Number of Folds)', fontsize=12)
plt.ylabel('Mean Accuracy', fontsize=12)
plt.title('K-Fold CV Performance vs K Value', fontsize=14, pad=20)
plt.grid(True, alpha=0.3)
plt.show()

print("\n=== K 값에 따른 성능 ===")
for k, mean, std in zip(k_values, mean_scores, std_scores):
    print(f"K={k:2d}: {mean:.4f} (+/- {std:.4f})")

### 1.2 Stratified K-Fold (계층화 K-Fold)

In [ ]:
# 클래스 비율 확인
unique, counts = np.unique(y, return_counts=True)
print("전체 데이터 클래스 분포:")
for cls, cnt in zip(unique, counts):
    print(f"  Class {cls}: {cnt} ({cnt/len(y)*100:.1f}%)")

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\n=== Stratified K-Fold 각 폴드의 클래스 분포 ===")
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    train_classes = np.bincount(y[train_idx])
    val_classes = np.bincount(y[val_idx])
    
    print(f"Fold {fold}:")
    print(f"  Train: {train_classes} ({train_classes/train_classes.sum()*100})")
    print(f"  Val:   {val_classes} ({val_classes/val_classes.sum()*100})")

In [ ]:
# K-Fold vs Stratified K-Fold 성능 비교
kf = KFold(n_splits=5, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_kf = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
scores_skf = cross_val_score(model, X, y, cv=skf, scoring='accuracy')

print("=== K-Fold vs Stratified K-Fold ===")
print(f"K-Fold:           {scores_kf.mean():.4f} (+/- {scores_kf.std():.4f})")
print(f"Stratified K-Fold: {scores_skf.mean():.4f} (+/- {scores_skf.std():.4f})")
print("\n불균형 데이터에서는 Stratified K-Fold가 더 안정적입니다.")

### 1.3 다양한 교차검증 방법

In [ ]:
# Leave-One-Out (LOO)
loo = LeaveOneOut()
print(f"Leave-One-Out 분할 수: {loo.get_n_splits(X)} (데이터 수와 동일)")
print("LOO는 작은 데이터셋에서 유용하지만 계산 비용이 높습니다.\n")

# Shuffle Split (랜덤 분할)
ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
scores_ss = cross_val_score(model, X, y, cv=ss, scoring='accuracy')
print(f"Shuffle Split 평균: {scores_ss.mean():.4f} (+/- {scores_ss.std():.4f})")
print("각 분할이 독립적으로 랜덤 샘플링됩니다.\n")

# Repeated K-Fold (반복)
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
scores_rkf = cross_val_score(model, X, y, cv=rkf, scoring='accuracy')
print(f"Repeated K-Fold 평균: {scores_rkf.mean():.4f} (+/- {scores_rkf.std():.4f})")
print(f"총 분할 수: {len(scores_rkf)} (5 folds × 10 repeats = 50)")
print("더 안정적인 추정을 위해 여러 번 반복합니다.")

### 1.4 시계열 교차검증 (Time Series Split)

In [ ]:
# 시계열 데이터용 교차검증
tscv = TimeSeriesSplit(n_splits=5)

print("=== Time Series Split ===")
print("시계열 데이터는 과거 → 미래 순서를 유지해야 합니다.\n")

for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
    print(f"Fold {fold}:")
    print(f"  Train: [{train_idx[0]:3d}:{train_idx[-1]:3d}] ({len(train_idx)} samples)")
    print(f"  Test:  [{test_idx[0]:3d}:{test_idx[-1]:3d}] ({len(test_idx)} samples)")

In [ ]:
# Time Series Split 시각화
fig, ax = plt.subplots(figsize=(12, 6))

for i, (train, test) in enumerate(tscv.split(X)):
    # Train set
    ax.barh(i, len(train), left=train[0], height=0.4, 
            align='center', color='blue', alpha=0.6, label='Train' if i == 0 else '')
    # Test set
    ax.barh(i, len(test), left=test[0], height=0.4, 
            align='center', color='red', alpha=0.6, label='Test' if i == 0 else '')

ax.set_yticks(range(tscv.n_splits))
ax.set_yticklabels([f'Fold {i+1}' for i in range(tscv.n_splits)])
ax.set_xlabel('Sample Index', fontsize=12)
ax.set_title('Time Series Split Visualization', fontsize=14, pad=20)
ax.legend(loc='upper left', fontsize=11)
plt.tight_layout()
plt.show()

## 2. cross_val_score vs cross_validate

In [ ]:
# cross_validate: 여러 지표 동시 평가
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']

cv_results = cross_validate(
    model, X, y,
    cv=5,
    scoring=scoring,
    return_train_score=True
)

print("=== cross_validate 결과 ===")
for metric in scoring:
    train_key = f'train_{metric}'
    test_key = f'test_{metric}'
    print(f"\n{metric}:")
    print(f"  Train: {cv_results[train_key].mean():.4f} (+/- {cv_results[train_key].std():.4f})")
    print(f"  Test:  {cv_results[test_key].mean():.4f} (+/- {cv_results[test_key].std():.4f})")

print(f"\n평균 학습 시간: {cv_results['fit_time'].mean():.4f}초")
print(f"평균 예측 시간: {cv_results['score_time'].mean():.4f}초")

In [ ]:
# 결과 시각화
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Train': [cv_results[f'train_{m}'].mean() for m in scoring],
    'Test': [cv_results[f'test_{m}'].mean() for m in scoring]
})

x = np.arange(len(metrics_df))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, metrics_df['Train'], width, label='Train', alpha=0.8)
bars2 = ax.bar(x + width/2, metrics_df['Test'], width, label='Test', alpha=0.8)

ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Train vs Test Scores (5-Fold CV)', fontsize=14, pad=20)
ax.set_xticks(x)
ax.set_xticklabels(metrics_df['Metric'])
ax.legend(fontsize=11)
ax.set_ylim([0.9, 1.0])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. 하이퍼파라미터 튜닝

### 3.1 Grid Search

In [ ]:
# Breast Cancer 데이터셋
cancer = load_breast_cancer()
X_cancer, y_cancer = cancer.data, cancer.target

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cancer)

# 하이퍼파라미터 그리드
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

print("=== Grid Search ===")
print(f"파라미터 조합 수: {len(param_grid['C']) * len(param_grid['gamma']) * len(param_grid['kernel'])}")
print(f"CV Folds: 5")
print(f"총 fit 횟수: {32 * 5} = 160\n")

In [ ]:
# Grid Search 실행
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1  # 모든 CPU 사용
)

grid_search.fit(X_scaled, y_cancer)

print("\n=== Grid Search 결과 ===")
print(f"최적 파라미터: {grid_search.best_params_}")
print(f"최적 점수: {grid_search.best_score_:.4f}")
print(f"최적 모델: {grid_search.best_estimator_}")

In [ ]:
# 모든 결과 확인
results_df = pd.DataFrame(grid_search.cv_results_)

# 상위 10개 조합
top_results = results_df.nsmallest(10, 'rank_test_score')[[
    'params', 'mean_test_score', 'std_test_score', 'rank_test_score'
]]

print("\n=== 상위 10개 파라미터 조합 ===")
for idx, row in top_results.iterrows():
    print(f"Rank {int(row['rank_test_score'])}: {row['params']}")
    print(f"  Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']:.4f})\n")

In [ ]:
# Grid Search 결과 히트맵 (C vs gamma, rbf kernel)
rbf_results = results_df[results_df['param_kernel'] == 'rbf']

# Pivot table 생성
pivot_table = rbf_results.pivot_table(
    values='mean_test_score',
    index='param_gamma',
    columns='param_C'
)

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt='.4f', cmap='YlGnBu', 
            cbar_kws={'label': 'Accuracy'})
plt.title('Grid Search Results (RBF Kernel): C vs Gamma', fontsize=14, pad=20)
plt.xlabel('C (Regularization Parameter)', fontsize=12)
plt.ylabel('Gamma', fontsize=12)
plt.tight_layout()
plt.show()

### 3.2 Randomized Search

In [ ]:
# 하이퍼파라미터 분포 정의
param_distributions = {
    'C': uniform(0.1, 100),  # 0.1 ~ 100.1 균등 분포
    'gamma': uniform(0.001, 1),  # 0.001 ~ 1.001 균등 분포
    'kernel': ['rbf', 'linear', 'poly']
}

# Randomized Search 실행
random_search = RandomizedSearchCV(
    SVC(random_state=42),
    param_distributions,
    n_iter=50,  # 50개 조합 시도
    cv=5,
    scoring='accuracy',
    random_state=42,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_scaled, y_cancer)

print("\n=== Randomized Search 결과 ===")
print(f"최적 파라미터: {random_search.best_params_}")
print(f"최적 점수: {random_search.best_score_:.4f}")

In [ ]:
# Grid Search vs Randomized Search 비교
comparison_df = pd.DataFrame({
    'Method': ['Grid Search', 'Randomized Search'],
    'Best Score': [grid_search.best_score_, random_search.best_score_],
    'N Iterations': [len(grid_search.cv_results_['params']), 
                     len(random_search.cv_results_['params'])]
})

print("\n=== Grid Search vs Randomized Search ===")
print(comparison_df.to_string(index=False))
print("\nRandomized Search:")
print("  - 장점: 계산 효율적, 연속 분포 탐색 가능")
print("  - 단점: 최적해 보장 없음")
print("\nGrid Search:")
print("  - 장점: 모든 조합 탐색, 최적해 보장 (그리드 내)")
print("  - 단점: 조합 수가 기하급수적으로 증가")

### 3.3 Random Forest 하이퍼파라미터 튜닝

In [ ]:
# Random Forest 파라미터 그리드
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    rf_param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

rf_grid_search.fit(X_cancer, y_cancer)

print("\n=== Random Forest Grid Search 결과 ===")
print(f"최적 파라미터: {rf_grid_search.best_params_}")
print(f"최적 점수: {rf_grid_search.best_score_:.4f}")

In [ ]:
# Feature Importance 시각화
best_rf = rf_grid_search.best_estimator_
feature_importance = pd.DataFrame({
    'feature': cancer.feature_names,
    'importance': best_rf.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'][:15], feature_importance['importance'][:15])
plt.xlabel('Importance', fontsize=12)
plt.title('Top 15 Feature Importances (Optimized Random Forest)', fontsize=14, pad=20)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 5 Features:")
print(feature_importance.head().to_string(index=False))

## 4. 중첩 교차검증 (Nested Cross-Validation)

In [ ]:
# 중첩 CV: 외부 루프(모델 평가) + 내부 루프(하이퍼파라미터 튜닝)

# 내부 CV (하이퍼파라미터 튜닝)
param_grid_nested = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01]}
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid_search_nested = GridSearchCV(
    SVC(kernel='rbf', random_state=42), 
    param_grid_nested, 
    cv=inner_cv, 
    scoring='accuracy'
)

# 외부 CV (모델 평가)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
nested_scores = cross_val_score(
    grid_search_nested, 
    X_scaled, 
    y_cancer, 
    cv=outer_cv, 
    scoring='accuracy'
)

print("=== 중첩 교차검증 결과 ===")
print(f"각 외부 폴드 점수: {nested_scores}")
print(f"평균 점수: {nested_scores.mean():.4f} (+/- {nested_scores.std():.4f})")

# 비교: 일반 CV vs 중첩 CV
grid_search_nested.fit(X_scaled, y_cancer)
print(f"\n일반 CV 최적 점수: {grid_search_nested.best_score_:.4f}")
print(f"중첩 CV 평균 점수: {nested_scores.mean():.4f}")
print("\n중첩 CV가 더 현실적인 일반화 성능을 추정합니다.")
print("일반 CV는 과대평가될 수 있습니다 (데이터 누수).")

## 5. 파이프라인과 함께 사용

In [ ]:
# 파이프라인 정의
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(random_state=42))
])

# 파라미터 이름: step__parameter
param_grid_pipeline = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': [0.1, 0.01, 0.001],
    'svm__kernel': ['rbf', 'linear']
}

grid_search_pipeline = GridSearchCV(
    pipeline, 
    param_grid_pipeline, 
    cv=5, 
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# 스케일링되지 않은 데이터 사용 (파이프라인이 처리)
grid_search_pipeline.fit(X_cancer, y_cancer)

print("\n=== 파이프라인 Grid Search 결과 ===")
print(f"최적 파라미터: {grid_search_pipeline.best_params_}")
print(f"최적 점수: {grid_search_pipeline.best_score_:.4f}")
print("\n파이프라인 사용의 장점:")
print("  - 전처리 단계를 자동으로 포함")
print("  - CV에서 데이터 누수 방지")
print("  - 코드 간결성")

## 6. 학습 곡선 (Learning Curves)

In [ ]:
# 학습 곡선 계산
train_sizes, train_scores, val_scores = learning_curve(
    grid_search_pipeline.best_estimator_,
    X_cancer, y_cancer,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# 평균 및 표준편차
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

# 학습 곡선 시각화
plt.figure(figsize=(10, 6))
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, 
                 alpha=0.2, color='blue')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, 
                 alpha=0.2, color='orange')
plt.plot(train_sizes, train_mean, 'o-', color='blue', linewidth=2, 
         label='Training Score')
plt.plot(train_sizes, val_mean, 'o-', color='orange', linewidth=2, 
         label='Validation Score')
plt.xlabel('Training Set Size', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Learning Curve', fontsize=14, pad=20)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("학습 곡선 해석:")
print("  - 두 곡선이 모두 낮음 → 과소적합")
print("  - 훈련 곡선 높고 검증 곡선 낮음 → 과적합")
print("  - 두 곡선이 수렴 → 적절한 적합")

## 7. 검증 곡선 (Validation Curve)

In [ ]:
# C 파라미터에 대한 검증 곡선
param_range = np.logspace(-4, 2, 10)

train_scores_val, test_scores_val = validation_curve(
    SVC(kernel='rbf', gamma=0.01, random_state=42),
    X_scaled, y_cancer,
    param_name='C',
    param_range=param_range,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

train_mean_val = train_scores_val.mean(axis=1)
train_std_val = train_scores_val.std(axis=1)
test_mean_val = test_scores_val.mean(axis=1)
test_std_val = test_scores_val.std(axis=1)

# 검증 곡선 시각화
plt.figure(figsize=(10, 6))
plt.semilogx(param_range, train_mean_val, 'o-', color='blue', linewidth=2, 
             label='Training Score')
plt.semilogx(param_range, test_mean_val, 'o-', color='orange', linewidth=2, 
             label='Validation Score')
plt.fill_between(param_range, train_mean_val - train_std_val, 
                 train_mean_val + train_std_val, alpha=0.2, color='blue')
plt.fill_between(param_range, test_mean_val - test_std_val, 
                 test_mean_val + test_std_val, alpha=0.2, color='orange')
plt.xlabel('C (Regularization Parameter)', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Validation Curve (SVM RBF)', fontsize=14, pad=20)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("검증 곡선 해석:")
print("  - 왼쪽(작은 C): 과소적합 (정규화 강함)")
print("  - 중간: 적절한 복잡도")
print("  - 오른쪽(큰 C): 과적합 가능성 (정규화 약함)")

## 8. 커스텀 스코어링 함수

In [ ]:
# 커스텀 스코어링 함수
def custom_f1_score(y_true, y_pred):
    """가중 평균 F1-score"""
    return f1_score(y_true, y_pred, average='weighted')

custom_scorer = make_scorer(custom_f1_score)

# 커스텀 스코어러 사용
scores_custom = cross_val_score(
    LogisticRegression(max_iter=1000, random_state=42), 
    X, y, 
    cv=5, 
    scoring=custom_scorer
)

print("=== 커스텀 스코어링 함수 ===")
print(f"커스텀 F1-score: {scores_custom.mean():.4f} (+/- {scores_custom.std():.4f})")

# 내장 스코어링 함수들
print("\n내장 스코어링 함수:")
print("  분류: 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'")
print("  회귀: 'r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'")

## 9. 결과 저장 및 로드

In [ ]:
import joblib
import json

# 최적 모델 저장
best_model = grid_search_pipeline.best_estimator_
joblib.dump(best_model, 'best_model.pkl')
print("최적 모델이 'best_model.pkl'에 저장되었습니다.")

# 결과 저장
results = {
    'best_params': grid_search_pipeline.best_params_,
    'best_score': grid_search_pipeline.best_score_,
    'cv_results': {
        k: v.tolist() if isinstance(v, np.ndarray) else v
        for k, v in grid_search_pipeline.cv_results_.items()
        if k in ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
    }
}

with open('tuning_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print("튜닝 결과가 'tuning_results.json'에 저장되었습니다.")

# 모델 로드
loaded_model = joblib.load('best_model.pkl')
print(f"\n로드된 모델: {loaded_model}")

## 요약

### 교차검증 방법 선택

| 기법 | 용도 | 특징 |
|------|------|------|
| K-Fold | 일반적인 평가 | 데이터를 K개로 분할 |
| Stratified K-Fold | 불균형 데이터 | 클래스 비율 유지 |
| Time Series Split | 시계열 데이터 | 시간 순서 유지 |
| Leave-One-Out | 작은 데이터셋 | 계산 비용 높음 |

### 하이퍼파라미터 튜닝 방법

| 방법 | 장점 | 단점 | 사용 시기 |
|------|------|------|----------|
| Grid Search | 완전 탐색 | 계산 비용 높음 | 파라미터 적고 범위 명확 |
| Randomized Search | 효율적 | 최적해 보장 없음 | 파라미터 많고 범위 불확실 |
| Nested CV | 신뢰성 높음 | 계산 비용 매우 높음 | 연구, 벤치마크 |

### 실전 팁

1. **작은 데이터셋**: Stratified K-Fold (k=5 or 10)
2. **큰 데이터셋**: Stratified K-Fold (k=3) 또는 단일 train/test split
3. **시계열**: Time Series Split
4. **파라미터 탐색**: Grid Search (좁은 범위) → Randomized Search (넓은 범위)
5. **파이프라인**: 전처리 포함하여 데이터 누수 방지